In [1]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

import os
import sys
sys.path.append('C:\\Users\\yu886\\OneDrive\\デスクトップ\\github\\Sony\\src')

from Processing.load_dataset import Load_dataset

from Engineering.dummy import engin_2, engin

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter('ignore', category=UserWarning)
import matplotlib.pyplot as plt
from Processing.processing import Submission

In [2]:
import optuna.integration.lightgbm as lgb

In [3]:
train, test = Load_dataset()

index = []
a = list(train[(train["co_min"]>=250)].index)
b = list(train[(train["co_mid"]>=200)].index)
c = list(train[(train["o3_mid"]>=200)].index)
d = list(train[(train["so2_min"]>=150)].index)
e = list(train[(train["so2_mid"]>=200)].index)
f = list(train[(train["no2_min"]>=100)].index)
g = list(train[(train["no2_mid"]>=400)].index)
h = list(train[(train["temperature_min"]>=40)].index)
i = list(train[(train["ws_mid"]>=40)].index)
j = list(train[(train["dew_var"]>=1400)].index)
index = a+b+c+d+e+f+g+h+i+j
index = list(set(index))
train.drop(index=index, axis=1)


df_train = engin_2(train)
df_test = engin_2(test)

df_train.drop(index=index, axis=1)

target = df_train["pm25_mid"]

df_train = df_train.drop(["id","pm25_mid"], axis=1)
df_test = df_test.drop(["id"], axis=1)

publisher_train = train['City']
unique_publisher = train['City'].unique()

In [ ]:
FOLD = 5
NUM_ROUND = 2000
params = {
    'objective': 'regression',
    'metric': 'rmse',
    "verbosity": -1,
    "boosting_type": "gbdt",
    'learning_rate':0.01
}

models = []

kf = KFold(n_splits=FOLD, shuffle=True, random_state=123)
for fold, (tr_group_idx, va_group_idx) in enumerate(kf.split(unique_publisher)):
    tr_groups, va_groups = unique_publisher[tr_group_idx], unique_publisher[va_group_idx]
    
    is_tr = publisher_train.isin(tr_groups)
    is_va = publisher_train.isin(va_groups)
    # ここでCityの情報を消したい
    
    X_train, X_valid = df_train[is_tr], df_train[is_va]
    y_train, y_valid = target[is_tr], target[is_va]
    
    X_train = X_train.drop("City" ,axis=1)
    X_valid = X_valid.drop("City" ,axis=1)
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=NUM_ROUND,
        verbose_eval=0
    )

    y_valid_pred = model.predict(X_valid)
    MSE = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    MAE = mean_absolute_error(y_valid, y_valid_pred)
    
    print(f'fold {fold} MSE: {MSE} RMSE: {np.sqrt(MSE)} R^2: {r2} MAE: {MAE}')
    models.append(model)

[I 2022-04-27 21:39:53,779] A new study created in memory with name: no-name-79501b83-5582-4a8c-adda-a2637f87a1d9


  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

feature_fraction, val_score: 21.036800:   0%|                                                    | 0/7 [00:43<?, ?it/s]

feature_fraction, val_score: 21.036800:  14%|######2                                     | 1/7 [00:43<04:23, 43.87s/it][I 2022-04-27 21:40:37,660] Trial 0 finished with value: 21.036799714156622 and parameters: {'feature_fraction': 0.6}. Best is trial 0 with value: 21.036799714156622.


feature_fraction, val_score: 21.036800:  14%|######2                                     | 1/7 [00:43<04:23, 43.87s/it]

feature_fraction, val_score: 21.030933:  14%|######2                                     | 1/7 [01:25<04:23, 43.

fold 0 MSE: 424.06230081794513 RMSE: 20.5927730239991 R^2: 0.6355742261463982 MAE: 14.991446949820073




feature_fraction, val_score: 22.087277:   0%|                                                    | 0/7 [00:11<?, ?it/s]

feature_fraction, val_score: 22.087277:  14%|######2                                     | 1/7 [00:11<01:06, 11.07s/it][I 2022-04-27 22:33:44,000] Trial 0 finished with value: 22.08727662493487 and parameters: {'feature_fraction': 0.4}. Best is trial 0 with value: 22.08727662493487.


feature_fraction, val_score: 22.087277:  14%|######2                                     | 1/7 [00:11<01:06, 11.07s/it]

feature_fraction, val_score: 22.087277:  14%|######2                                     | 1/7 [00:23<01:06, 11.07s/it]

feature_fraction, val_score: 22.087277:  29%|############5                               | 2/7 [00:23<01:00, 12.04s/it][I 2022-04-27 22:33:56,721] Trial 1 finished with value: 22.135021073738596 and parameters: {'feature_fraction': 0.5}. Best is trial 0 with value: 22.08727662493487.


feature_fraction, val_score: 22.087277:  29%|############5    

fold 1 MSE: 470.1442760822141 RMSE: 21.682810613068916 R^2: 0.6698440934028111 MAE: 15.73431270200023




feature_fraction, val_score: 21.708928:   0%|                                                    | 0/7 [00:10<?, ?it/s]

feature_fraction, val_score: 21.708928:  14%|######2                                     | 1/7 [00:10<01:03, 10.65s/it][I 2022-04-27 23:26:28,065] Trial 0 finished with value: 21.70892788939518 and parameters: {'feature_fraction': 0.7}. Best is trial 0 with value: 21.70892788939518.


feature_fraction, val_score: 21.708928:  14%|######2                                     | 1/7 [00:10<01:03, 10.65s/it]

feature_fraction, val_score: 21.644359:  14%|######2                                     | 1/7 [00:20<01:03, 10.65s/it]

feature_fraction, val_score: 21.644359:  29%|############5                               | 2/7 [00:20<00:50, 10.09s/it][I 2022-04-27 23:26:37,764] Trial 1 finished with value: 21.644359014701486 and parameters: {'feature_fraction': 0.4}. Best is trial 1 with value: 21.644359014701486.


feature_fraction, val_score: 21.644359:  29%|############5   

fold 2 MSE: 447.8197136020622 RMSE: 21.16175119412527 R^2: 0.659545074566197 MAE: 15.230034608013584




feature_fraction, val_score: 22.717769:   0%|                                                    | 0/7 [00:20<?, ?it/s]

feature_fraction, val_score: 22.717769:  14%|######2                                     | 1/7 [00:20<02:02, 20.46s/it][I 2022-04-28 00:11:24,613] Trial 0 finished with value: 22.717769450764997 and parameters: {'feature_fraction': 0.7}. Best is trial 0 with value: 22.717769450764997.


feature_fraction, val_score: 22.717769:  14%|######2                                     | 1/7 [00:20<02:02, 20.46s/it]

feature_fraction, val_score: 22.717769:  14%|######2                                     | 1/7 [00:37<02:02, 20.46s/it]

feature_fraction, val_score: 22.717769:  29%|############5                               | 2/7 [00:37<01:32, 18.43s/it][I 2022-04-28 00:11:41,625] Trial 1 finished with value: 22.74665308269146 and parameters: {'feature_fraction': 0.5}. Best is trial 0 with value: 22.717769450764997.


feature_fraction, val_score: 22.717769:  29%|############5  

In [10]:
models[0]

In [11]:
best_params = models[0].params
best_params

{'objective': 'mean_squared_error',
 'metric': 'l1',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 0.048278664038808795,
 'lambda_l2': 4.662554533983178e-07,
 'num_leaves': 35,
 'feature_fraction': 0.7,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 2000,
 'early_stopping_round': None}

In [18]:
# df_test = df_test.drop('City', axis=1)
y_pred = models[0].predict(df_test, num_iteration=models[0].best_iteration)
sum(y_pred)

3122626.8072049743

In [20]:
index = test["id"]
Submission(index, y_pred, name='lgb_optuna_2')